In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
import lightgbm as ltb

from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

import plotly.graph_objects as go
import plotly.express as px

#### NationalityId, PositionId, FootId, TeamId, LeagueId
Those columns contains categorical values, in order to not create an 'importance' scale based on their ID value, we need to encode them

- how can create numerical features without one-hot encoding
    - For : Nationality, Position, Foot, Team, League
        - y_level_mean = x.replace(y.groupby(x).mean()) 
        - JamesSteinEncoder
    - For Foot only
        -  WOEEncoder (for the foot)

In [23]:
def compute_numerical_feature_from_categorical(dataset, y, categorical_feature_name, numerical_feature_name):

    # Calculate the mean of the target variable grouped by the categorical feature
    feature_mean_value = dataset.groupby(categorical_feature_name)[y].mean().reset_index()

    # Rename the aggregated column to 'numerical_feature_name'
    feature_mean_value = feature_mean_value.rename(columns={y: numerical_feature_name})

    # Join the mean values back to the original dataset
    dataset = dataset.merge(feature_mean_value, on=categorical_feature_name, how='left')

    # Drop the original categorical feature and any duplicate columns that may have been created during the join
    dataset = dataset.drop(columns=[categorical_feature_name])

    return dataset, feature_mean_value

In [24]:
def apply_encoder(dataset, encoder, categorical_feature_name, numerical_feature_name):
    dataset = dataset.merge(encoder, on=categorical_feature_name, how='left')
    dataset[numerical_feature_name] = dataset[numerical_feature_name].fillna(dataset[numerical_feature_name].mean())
    dataset = dataset.drop(columns=[categorical_feature_name])

    return dataset

def get_player_name(player_ids, player_id_name):
    player_ids = pd.DataFrame(player_ids)
    player_name = player_ids \
        .merge(player_id_name, on='PlayerId', how='left') \
        .drop('PlayerId', axis=1)

    return player_name

def plot(df):
    # Residuals Plot
    fig_residuals = px.scatter(df, x='Predicted Salary', y='Residuals', hover_data=['PlayerName'])
    fig_residuals.update_layout(title='Residuals vs Predicted Salary', xaxis_title='Predicted Salary', yaxis_title='Predicted - Actual')

    # Prediction vs Actual Plot
    fig_pred_vs_actual = px.scatter(df, x='Actual Salary', y='Predicted Salary', hover_data=['PlayerName'])
    fig_pred_vs_actual.add_trace(go.Scatter(x=[df['Actual Salary'].min(), df['Actual Salary'].max()],
                                            y=[df['Actual Salary'].min(), df['Actual Salary'].max()],
                                            mode='lines', line=dict(dash='dash'), name='Perfect Prediction'))
    fig_pred_vs_actual.update_layout(title='Predicted vs Actual Salary', xaxis_title='Actual Salary', yaxis_title='Predicted Salary')

    # Histogram of Residuals
    fig_hist_residuals = px.histogram(df, x='Residuals')
    fig_hist_residuals.update_layout(title='Histogram of Residuals', xaxis_title='Residuals', yaxis_title='Count')

    # Show the plots
    fig_residuals.show()
    fig_pred_vs_actual.show()
    fig_hist_residuals.show()

def compute_metrics(y, y_pred):
    # Assuming predictions_df is your dataframe
    # Mean Absolute Error (MAE)
    mae = metrics.mean_absolute_error(y, y_pred)
    # Mean Squared Error (MSE)
    mse = metrics.mean_squared_error(y, y_pred)
    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)  # Or directly, metrics.mean_squared_error(y_test, y_pred, squared=False)
    # R-squared (R²)
    r2 = metrics.r2_score(y, y_pred)

    return mae, rmse, r2

def get_metrics(name_model, predictions_df, y_train, y_train_pred):

    mae_test, rmse_test, r2_test = compute_metrics(predictions_df['Actual Salary'], predictions_df['Predicted Salary'])
    mae_train, rmse_train, r2_train = compute_metrics(y_train, y_train_pred)

    computed_metrics = {
        'Model': [name_model],
        'Test - MAE': [mae_test],
        'Test - RMSE': [rmse_test],
        'Test - R-squared': [r2_test],
        'Train - MAE': [mae_train],
        'Train - RMSE': [rmse_train],
        'Train - R-squared': [r2_train]
    }

    computed_metrics = pd.DataFrame(computed_metrics)

    return computed_metrics

def train_test_score_model(model_name, model, scaler, player_names, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)

    # Transform the testing data (using the same scaler)
    y_pred = model.predict(scaler.transform(x_test))
    # Assuming 'predictions' is your array of predictions from the trained model
    adjusted_y_pred = np.maximum(y_pred, 0)

    a = len(player_names['PlayerName'].to_list())
    b = len([model_name for i in range(len(adjusted_y_pred))])
    c = len(list(y_test))
    d = len(list(adjusted_y_pred))
    e = len(list(adjusted_y_pred - y_test))
    f = len(list(adjusted_y_pred - y_test)/y_test*100)

    # Create a DataFrame
    predictions_df = pd.DataFrame({
        'PlayerName': player_names['PlayerName'].to_list(),
        'Model': [model_name for i in range(len(adjusted_y_pred))],
        'Actual Salary': list(y_test),
        'Predicted Salary': list(adjusted_y_pred),
        'Residuals': list(adjusted_y_pred - y_test),
        'Percentage Error' : list(adjusted_y_pred - y_test)/y_test*100
    })

    y_pred_train = model.predict(x_train)
    adjusted_y_pred_train = np.maximum(y_pred_train, 0)
    computed_metrics = get_metrics(model_name, predictions_df, y_train, adjusted_y_pred_train)

    return model, predictions_df, computed_metrics

## Load dataset

In [25]:
dataset = pd.read_csv('gold/dataset.csv')
player_name_df = pd.read_csv('silver/PlayerName.csv')

## Training, Validation, Testing split

In [26]:
col_to_drop = ['Wage']
X = dataset.drop(col_to_drop, axis=1)  # Assuming 'Value' is the target
y = dataset['Wage']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Process training and testing set

In [28]:
x_train = x_train.drop('PlayerId', axis=1)
test_player_ids = x_test['PlayerId']
player_names = get_player_name(test_player_ids, player_name_df)
x_test = x_test.drop('PlayerId', axis=1)

In [29]:
# encode categorical columns
x_train, nationality_encoder = compute_numerical_feature_from_categorical(x_train, 'Value', 'NationalityId', 'Nationality')
x_train, position_encoder = compute_numerical_feature_from_categorical(x_train, 'Value', 'PositionId', 'Position')
x_train, foot_encoder = compute_numerical_feature_from_categorical(x_train, 'Value', 'FootId', 'Foot')
x_train, team_encoder = compute_numerical_feature_from_categorical(x_train, 'Value', 'TeamId', 'Team')
x_train, league_encoder = compute_numerical_feature_from_categorical(x_train, 'Value', 'LeagueId', 'League')

x_test = apply_encoder(x_test, nationality_encoder, 'NationalityId', 'Nationality')
x_test = apply_encoder(x_test, position_encoder, 'PositionId', 'Position')
x_test = apply_encoder(x_test, foot_encoder, 'FootId', 'Foot')
x_test = apply_encoder(x_test, team_encoder, 'TeamId', 'Team')
x_test = apply_encoder(x_test, league_encoder, 'LeagueId', 'League')


scaler_type = 'MinMaxScaler'
if scaler_type == 'MinMaxScaler': scaler = MinMaxScaler(feature_range=(0, 1))
else: scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(x_train)

# Transform the training data
x_train_scaled = scaler.transform(x_train)

## Train - Test - Score Models

In [31]:
models = {
    'Linear Regressor': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(n_estimators=300, max_depth=30, random_state=42),
    'LightGBM': ltb.LGBMRegressor(colsample_bytree=0.7, learning_rate=0.1, max_depth=10, min_child_samples=20, n_estimators=300, num_leaves=31, subsample=0.8, random_state=42, verbose=-1),
    }

# Ridge cv
# Light gbm
# XG boost
# Any estimator using the Huber loss would also be robust to outliers, e.g. SGDRegressor with loss='huber'.

prediction_df = [] # pd.DataFrame(columns=['Player', 'Model', 'Actual Salary', 'Predicted Salary', 'Residuals'])
computed_metrics = [] # pd.DataFrame(columns=['Model', 'MAE', 'MSE', 'RMSE', 'R-squared', 'Adjusted R-squared'])

for name, model in models.items():
    trained_model, pred_df, metric_df = train_test_score_model(name, model, scaler, player_names, x_train_scaled, y_train, x_test, y_test)
    models[name] = trained_model
    prediction_df.append(pred_df)
    computed_metrics.append(metric_df)

prediction_df = pd.concat(prediction_df)
computed_metrics = pd.concat(computed_metrics)

ValueError: All arrays must be of the same length

In [ ]:
computed_metrics

,Model,Test - MAE,Test - RMSE,Test - R-squared,Train - MAE,Train - RMSE,Train - R-squared
0,Linear Regressor,6768.662575,13418.278795,0.747218,6415.752532,11731.782080,0.794727
0,Random Forest Regressor,4159.433805,8563.151276,0.897052,1481.338177,3123.687571,0.985447
0,LightGBM,3512.014061,7585.279981,0.919221,1436.884805,2382.378230,0.991535


In [ ]:
plot(prediction_df[prediction_df.Model == 'LightGBM'])

## Feature selection

In [ ]:
# Get feature importances
importances = models['Random Forest Regressor'].feature_importances_

# Match feature names (assuming you have a list of names in feature_names)
feature_names = list(x_train.columns)  # replace with your actual feature names

features = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)

features_importance = pd.DataFrame(features, columns=['Feature', 'Importance'])

In [ ]:
# Sort features by importance
sorted_indices = np.argsort(importances)[::-1]
sorted_feature_names = [feature_names[idx] for idx in sorted_indices]
sorted_importances = importances[sorted_indices]

# Create Plotly bar chart
fig = go.Figure([go.Bar(x=sorted_feature_names, y=sorted_importances)])
fig.update_layout(
    title='Feature Importances from Random Forest Regressor',
    xaxis_title='Features',
    yaxis_title='Importance',
    xaxis_tickangle=-45
)
fig.show()

## process prediction_df

In [ ]:
player_df = pd.read_csv('silver\Player.csv')
player_team_df = pd.read_csv('silver\PlayerTeam.csv')
team_league_df = pd.read_csv('silver\TeamLeague.csv')

team_df = pd.read_csv('silver\Team.csv')
league_df = pd.read_csv('silver\League.csv')
nationality_df = pd.read_csv('silver/Nationality.csv')
position_df = pd.read_csv('silver\Position.csv')

In [ ]:
prediction_df.head()

,PlayerName,Model,Actual Salary,Predicted Salary,Residuals,Percentage Error
1142,Jacopo Fazzini,Linear Regressor,3000,13054.123337,10054.123337,335.137445
4567,Nathan Tella,Linear Regressor,36000,35949.654242,-50.345758,-0.139849
2592,Janik Haberer,Linear Regressor,32000,25882.685220,-6117.314780,-19.116609
1677,Cristiano Piccini,Linear Regressor,2000,13334.496620,11334.496620,566.724831
4958,Koen Van Langendonck,Linear Regressor,2000,6249.862289,4249.862289,212.493114


In [ ]:
prediction_df = prediction_df[prediction_df.Model == 'LightGBM']

prediction_df = prediction_df \
        .merge(player_name_df, on='PlayerName', how='left') \
        .merge(player_df, on='PlayerId', how='left') \
        .merge(player_team_df, on='PlayerId', how='left') \
        .merge(team_league_df, on='TeamId', how='left')

prediction_df = prediction_df \
        .merge(nationality_df, on='NationalityId', how='left') \
        .merge(position_df, on='PositionId', how='left') \
        .merge(league_df, on='LeagueId', how='left') \
        .merge(team_df, on='TeamId', how='left')

prediction_df = prediction_df[['PlayerName', 'PlayerId', 'Model', 'Actual Salary',
                               'Predicted Salary','Residuals', 'Percentage Error',
                               'Nationality', 'Position', 'Team', 'League']]

prediction_df.head()

,PlayerName,PlayerId,Model,Actual Salary,Predicted Salary,Residuals,Percentage Error,Nationality,Position,Team,League
0,Jacopo Fazzini,3201,LightGBM,3000,9899.460120,6899.460120,229.982004,Italy,SUB,Empoli,Serie A
1,Nathan Tella,3551,LightGBM,36000,36327.603004,327.603004,0.910008,Nigeria,SUB,Bayer 04 Leverkusen,Bundesliga
2,Janik Haberer,3687,LightGBM,32000,30943.746999,-1056.253001,-3.300791,Germany,Attacking Midfield,FC Union Berlin,Bundesliga
3,Cristiano Piccini,6295,LightGBM,2000,3681.768234,1681.768234,84.088412,Italy,SUB,Sampdoria,Serie B
4,Koen Van Langendonck,4471,LightGBM,2000,3373.595053,1373.595053,68.679753,Belgium,RES,Westerlo,Pro League


## Save datasets

In [107]:
prediction_df.to_csv('gold\Prediction.csv')
features_importance.to_csv('gold\FeatureImportance.csv')